# Chaos

Napisz program, który będzie rysował trajektorię dla któregoś z chaotycznych atraktorów: Lorenza, Roeslera, Kuramoto-Shivashinsky'ego (lub innych znalezionych). Rozwiązanie wymaga rozwiązania równania różniczkowego. W Pythonie można to zrobić np. funkcją [`odeint` z pakietu SciPy](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.integrate.odeint.html#scipy.integrate.odeint).

Wielkim plusem będzie interaktywność programu: warto zbadać, czy układ ma chaotyczne właściwości dla wszystkich zestawów parametrów, czy tylko dla pewnego ich podzbioru.

In [133]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from scipy.integrate import odeint
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline
from pylab import rcParams
rcParams['figure.figsize'] = 14, 12
import ipywidgets as widgets
from IPython.display import display
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual

## Lorenz system
[Wikipedia](https://en.wikipedia.org/wiki/Lorenz_system)
$$
{\begin{aligned}{\frac {\mathrm {d} x}{\mathrm {d} t}}&=\sigma (y-x),\\{\frac {\mathrm {d} y}{\mathrm {d} t}}&=x(\rho -z)-y,\\{\frac {\mathrm {d} z}{\mathrm {d} t}}&=xy-\beta z.\end{aligned}}
$$

In [134]:
def draw_lorenz(rho, sigma, beta):
    def f(state, t):
      x, y, z = state  # unpack the state vector
      return sigma * (y - x), x * (rho - z) - y, x * y - beta * z  # derivatives

    state0 = [1.0, 1.0, 1.0]
    t = np.arange(0.0, 40.0, 0.01)

    states = odeint(f, state0, t)

    fig = plt.figure()
    ax = fig.gca(projection='3d')
    ax.plot(states[:,0], states[:,1], states[:,2])
    plt.show()

In [135]:
def interact_lorenz():
    def f(a, b, c):
        draw_lorenz(a, b, c)
        return
    
    a = widgets.FloatSlider(value=28.0, description='$\\rho$:', min=0, max=56, step=0.1, continuous_update=False)
    b = widgets.FloatSlider(value=10.0, description='$\\sigma$:', min=0, max=20, step=0.1, continuous_update=False)
    c = widgets.FloatSlider(value=8.0 / 3.0, description='$\\beta$:', min=0, max=10, step=0.01, continuous_update=False)
    ui = widgets.HBox([a, b, c])
    
    out = widgets.interactive_output(f, {'a': a, 'b': b, 'c': c})
    display(ui, out)

interact_lorenz()

Output()

## Rössler attractor
[Wikipedia](https://en.wikipedia.org/wiki/R%C3%B6ssler_attractor)

$$
{\begin{aligned}{\frac {\mathrm {d} x}{\mathrm {d} t}}&=-y-z,\\{\frac {\mathrm {d} y}{\mathrm {d} t}}&=x+ ay,\\{\frac {\mathrm {d} z}{\mathrm {d} t}}&=b+z(x-c).\end{aligned}}
$$

In [136]:
def draw_rossler(a, b, c):
    def f(state, t):
      x, y, z = state  # unpack the state vector
      return (- y - z), x + a * y, b + z * (x - c)  # derivatives
    
    state0 = [1.0, 1.0, 1.01]
    t = np.arange(0.0, 150.0, 0.01)
    
    states = odeint(f, state0, t)
    
    fig = plt.figure()
    ax = fig.gca(projection='3d')
    ax.plot(states[:,0], states[:,1], states[:,2])
    plt.show()

In [137]:
def interact_rossler():
    def f(a, b, c):
        draw_rossler(a, b, c)
        return
    
    a = widgets.FloatSlider(value=0.2, description='$a$:', min=-2, max=1, step=0.01, continuous_update=False)
    b = widgets.FloatSlider(value=0.2, description='$b$:', min=0.01, max=2, step=0.01, continuous_update=False)
    c = widgets.FloatSlider(value=5.7, description='$c$:', min=1, max=40, step=0.01, continuous_update=False)
    ui = widgets.HBox([a, b, c])
    
    out = widgets.interactive_output(f, {'a': a, 'b': b, 'c': c})
    display(ui, out)

interact_rossler()

Output()

## Lorenz 96 model
[Wikipedia](https://en.wikipedia.org/wiki/Lorenz_96_model)
$$ N - \text{Number of variables} $$
$$ F - \text{Forcing constant} $$

In [138]:
def draw_lorenz96(N, F):
    def Lorenz96(x,t):
      # compute state derivatives
      d = np.zeros(N)
      # first the 3 edge cases: i=1,2,N
      d[0] = (x[1] - x[N-2]) * x[N-1] - x[0]
      d[1] = (x[2] - x[N-1]) * x[0]- x[1]
      d[N-1] = (x[0] - x[N-3]) * x[N-2] - x[N-1]
      # then the general case
      for i in range(2, N-1):
          d[i] = (x[i+1] - x[i-2]) * x[i-1] - x[i]
      # add the forcing term
      d = d + F

      # return the state derivatives
      return d

    x0 = F*np.ones(N) # initial state (equilibrium)
    x0[19] += 0.01 # add small perturbation to 20th variable
    t = np.arange(0.0, 40.0, 0.01)

    x = odeint(Lorenz96, x0, t)

    # plot first three variables
    fig = plt.figure()
    ax = fig.gca(projection='3d')
    ax.plot(x[:,0],x[:,1],x[:,2])
    plt.show()

In [139]:
def interact_lorenz96():
    def f(a, b):
        draw_lorenz96(a, b)
        return
    
    a = widgets.IntSlider(value=36, description='$N$:', min=20, max=100, continuous_update=False)
    b = widgets.IntSlider(value=8, description='$F$:', min=0, max=40, continuous_update=False)
    ui = widgets.HBox([a, b])
    
    out = widgets.interactive_output(f, {'a': a, 'b': b})
    display(ui, out)

interact_lorenz96()

Output()